## 2. Tool Calling
Lets discuss about "How do agents share information with other agents?"

- Till now, we used LLMs to make a decision by choosing from different pipelines. 
- Now lets move ahead by demonstrating how to use an LLM to pick a function to execute and infer an argument to pass through the function. 
- The ability of LLMs to take actions and interact with their external environment is crucial, and a good interface for the LLMs is necessary to make this possible. 
- Tool calling is a key component of this process.
- We use LLMs for tool calling in a basic RAG pipeline and show how to make LLMs pick the best query pipeline to answer user queries.
- This process allows LLMs to not only pick a function to execute but also to infer the arguments needed. 
- This results in users getting more precise answers than standard RAG techniques.

In [5]:
"""
!pip3 uninstall llama-index
!pip3 install llama-index --upgrade --no-cache-dir --force-reinstall
"""

'\n!pip3 uninstall llama-index\n!pip3 install llama-index --upgrade --no-cache-dir --force-reinstall\n'

In [6]:
from dotenv import load_dotenv
import nest_asyncio
import os
nest_asyncio.apply()
load_dotenv()

# Access variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

**Define a Simple Tool**

Example functions like an `add()` function and a `mystery()` function are defined to illustrate tool calling.
The function tool wraps any given Python function, taking in both the add function and the mystery function with *type annotations* and *docstrings* used as prompts for the LLM.


In [7]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

 Lets import the LLM module and call the `predict_and_call()` function. 
 - It involves taking in a set of tools and an input prompt string or chat messages, allowing the LLM to decide which tool to call and execute it to get the final response. 
 - An example shows the intermediate steps of calling a function with specific arguments and getting the correct output.

In [8]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


### 2. **Define an Auto-Retrieval Tool**

Now let's define a more sophisticated agentic layer on top of vector search. 
- The LLM can choose vector search and infer metadata filters, which helps return more precise search results. 
- Using the same paper, GPT2, the lesson focuses on nodes or chunks, demonstrating how metadata attached to chunks, such as page labels, can be used to filter search results.

**Steps:**
- The process involves using a simple directory reader from LlamaIndex to load the parsed representation of a PDF file, splitting documents into chunks, and looking at the content and metadata of an example chunk. 
- A vector store index is defined over these nodes, building a RAG indexing pipeline. 
- The lesson shows how to query this RAG pipeline via metadata filters, specifying filters like page labels to get precise search results.



In [9]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["language_models_are_unsupervised_multitask_learners.pdf"]).load_data()

In [10]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: language_models_are_unsupervised_multitask_learners.pdf
file_path: language_models_are_unsupervised_multitask_learners.pdf
file_type: application/pdf
file_size: 582775
creation_date: 2024-06-07
last_modified_date: 2024-06-07

Language Models are Unsupervised Multitask Learners
Alec Radford*1Jeffrey Wu*1Rewon Child1David Luan1Dario Amodei**1Ilya Sutskever**1
Abstract
Natural language processing tasks, such as ques-
tion answering, machine translation, reading com-
prehension, and summarization, are typically
approached with supervised learning on task-
speciﬁc datasets. We demonstrate that language
models begin to learn these tasks without any ex-
plicit supervision when trained on a new dataset
of millions of webpages called WebText. When
conditioned on a document plus questions, the an-
swers generated by the language model reach 55
F1 on the CoQA dataset - matching or exceeding
the performance of 3 out of 4 baseline systems
without using the 127,000+ training

In [11]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [12]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of GPT2?", 
)

print(str(response))

GPT-2 demonstrates the ability of language models to perform a wide range of tasks in a zero-shot setting, achieving promising, competitive, and state-of-the-art results depending on the task.


Lets look at the source nodes to observe the meta data used. 

In [13]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}
{'page_label': '2', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}


**Defining a Auto Retrieval Tool**
Finally lets wraps the retrieval tool into a function that takes in both the query string and page numbers as filters in the same manner we saw in example. 
- The LLM can infer page numbers to filter for a user query, eliminating the need for manual metadata specification. 
- Metadata is not limited to page numbers and can include various elements like 
  - section IDs, 
  - headers, and 
  - footers through LlamaIndex abstractions. 
- The ability to use multiple metadata filters is especially prominent in advanced models like GPT-4.

In [14]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [15]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the high-level results of MetaGPT as described on page 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT demonstrates the ability of language models to perform a wide range of downstream tasks in a zero-shot setting without any parameter or architecture modification. It showcases promising, competitive, and state-of-the-art results across various tasks, highlighting the potential of language models for multitask learning.


In [16]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}
{'page_label': '2', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}


**Let's add some other tools!**

In [17]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [18]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are the MetaGPT comparisons with ChatDev described on page 8?", 
    verbose=True
)

for n in response.source_nodes:
    print(n.metadata)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT comparisons with ChatDev are not directly discussed in the provided context. The context mainly focuses on language models, data overlap analysis, generalization versus memorization, and the impact of text similarity on model performance.
{'page_label': '8', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}
{'page_label': '8', 'file_name': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_path': 'language_models_are_unsupervised_multitask_learners.pdf', 'file_type': 'application/pdf', 'file_size': 582775, 'creation_date': '2024-06-07', 'last_modified_date': '2024-06-07'}


In [19]:

response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "The paper discusses the impact of climate change on biodiversity and ecosystems."}
=== Function Output ===
The paper does not discuss the impact of climate change on biodiversity and ecosystems.
